### Generate data

In [ ]:
import pyvacon.analytics as analytics
import pyvacon.tools.converter as converter
import pyvacon.models.converter as model_converter
import pyvacon.models.tools as model_tools
import numpy as np
import random
import math

main_dir = 'C:\\ml_repos\\orig_data_sc'

simtimes_p = range(1,3*365)
refdate = analytics.ptime(2017, 1, 1, 0, 0, 0)
num_threads = 2

nsims = 10000 # number of sims to compute the call prices
n_samples = 200000 # number of samples for call prices

ntimesteps_per_year = 365

spot = 1.0
n_strikes_per_sample = 10 #number of strikes call prices are computed at each sample
expiry_intervalls =  [(5,180) , (181, 365),(366, 366+180), (366+180+1,2*365), (2*365+1,3*365)]
n_expiries_per_sample = len(expiry_intervalls)

n_overall_samples = n_samples *n_strikes_per_sample*n_expiries_per_sample
X_result = np.empty((n_overall_samples, 5 + 2))
Y_result = np.empty((n_overall_samples, 1))
variance_min = math.log(0.05)
variance_max = math.log(1.2)
counter = 0
for i in range(n_samples):
    if i % 5000 == 0:
        print('sample ' + str(i))
    initial_variance = random.uniform(variance_min, variance_max)
    speed_of_meanreversion = random.uniform(0.1, 5.0)
    long_run_variance = random.uniform(variance_min, variance_max)
    vol_of_variance = random.uniform(0.01, 1.2)
    correlation = random.uniform(-0.9,0.9)
    model = analytics.ScottChesneyModel('t', refdate, spot, initial_variance, speed_of_meanreversion, long_run_variance, vol_of_variance, correlation)

    tmp = np.random.random_integers(10,3*365, n_expiries_per_sample)
    simtimes_p = []
    for d in expiry_intervalls:
        simtimes_p.append( int(np.random.randint(d[0],d[1])) )
    simtimes = converter.createPTimeList(refdate, simtimes_p)

    tmp = analytics.ModelLab(model, refdate)
    seed = np.random.randint(1,1024)
    #print('sample ' + str(i) + '  seed ' + str(seed))
    tmp.simulate(simtimes, nsims,ntimesteps_per_year, num_threads, seed)
    for t in range(len(simtimes_p)):
        expiry_index = simtimes_p[t]
        expiry_yf = (expiry_index+1)/365.0
        strikes = np.random.uniform(0.5,1.5, n_strikes_per_sample)
        prices = model_tools.compute_statistics(tmp, strikes, t, 0, lambda x, y: max(x-y,0.0))
        #put_prices = model_tools.compute_statistics(tmp, strikes_puts, expiry_index, 0, lambda x, y: max(y-x,0.0))
        #print(put_prices)
        for k in range(n_strikes_per_sample):
            X_result[counter, 0] = initial_variance
            X_result[counter, 1] = speed_of_meanreversion
            X_result[counter, 2] = long_run_variance
            X_result[counter, 3] = vol_of_variance
            X_result[counter, 4] = correlation
            X_result[counter, 5] = strikes[k]
            X_result[counter, 6] = expiry_yf
            Y_result[counter, 0] = prices[k]; #analytics.calcImpliedVol(call_prices[j], strikes_calls[j], expiry_yf, 1.0, 1.0, 'C')
            counter = counter + 1
    if (i+1) % 500 == 0:
        np.save(main_dir+'\\sc_params_' + str(counter) + '.npy', X_result[0:counter,:])
        np.save(main_dir+'\\prices_' + str(counter) + '.npy', Y_result[0:counter,:])

#print(Y_result)
np.save(main_dir + '\\sc_params.npy', X_result)
np.save(main_dir + '\\prices.npy', Y_result)


sample 0


C:\Users\Anwender\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning: This function is deprecated. Please call randint(10, 1095 + 1) instead


sample 5000
sample 10000
sample 15000
sample 20000
